# Raw data exploration

### Imports

In [2]:
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

from src.utils.const import DATA_DIR, DROP

### Useful path to data

In [3]:
ROOT_DIR = os.path.join(os.getcwd(), '..')
RAW_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'raw')
INTERIM_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'interim')

## Generic useful function

In [4]:
from src.utils.wrapper import drop, drop_na, rename, reset_index, fill_na, convert_to, extract_stat_feature, apply

### movies.csv

In [5]:
movies = pd.read_csv(os.path.join(RAW_DIR, 'movies.csv'), encoding='utf-8',
                     dtype={'title': 'string', 'genres': 'category'})
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   movieId  58098 non-null  int64   
 1   title    58098 non-null  string  
 2   genres   58098 non-null  category
dtypes: category(1), int64(1), string(1)
memory usage: 1.1 MB


#### Specific movies functions used in pipe

In [6]:
def extract_year_from_title(df: pd.DataFrame) -> pd.DataFrame:
    regex = '.*\((\d{4})\).*'
    df['year'] = df['title'].str.extract(pat=regex, expand=False)
    return df


def extract_title_length(df: pd.DataFrame) -> pd.DataFrame:
    df['title_length'] = df['title'].str.len()
    return df


def encode_genre(df: pd.DataFrame) -> pd.DataFrame:
    genres = df['genres'].str.split('|')
    mlb = MultiLabelBinarizer()
    encoded_genre = pd.DataFrame(
        mlb.fit_transform(genres),
        index=df['movieId'],
        columns=mlb.classes_
    )
    df = pd.merge(df, encoded_genre, on='movieId', how='inner')
    return df


def remove_no_genres(df: pd.DataFrame) -> pd.DataFrame:
    df_no_genre = df[df['(no genres listed)'] == 1].index
    df.drop(index=df_no_genre, inplace=True)
    if DROP:
        print(f'Number of films without any genres to be dropped: {df_no_genre.shape[0]}')
    return df

#### Pipe movies

In [7]:
movies = movies. \
    pipe(extract_year_from_title). \
    pipe(convert_to, 'year', 'float32'). \
    pipe(fill_na, 'year', True). \
    pipe(extract_title_length). \
    pipe(encode_genre). \
    pipe(remove_no_genres). \
    pipe(drop, ['title', 'genres'])

movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53832 entries, 0 to 58097
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             53832 non-null  int64  
 1   year                53832 non-null  float32
 2   title_length        53832 non-null  Int64  
 3   (no genres listed)  53832 non-null  int32  
 4   Action              53832 non-null  int32  
 5   Adventure           53832 non-null  int32  
 6   Animation           53832 non-null  int32  
 7   Children            53832 non-null  int32  
 8   Comedy              53832 non-null  int32  
 9   Crime               53832 non-null  int32  
 10  Documentary         53832 non-null  int32  
 11  Drama               53832 non-null  int32  
 12  Fantasy             53832 non-null  int32  
 13  Film-Noir           53832 non-null  int32  
 14  Horror              53832 non-null  int32  
 15  IMAX                53832 non-null  int32  
 16  Musi

#### Save movies.csv to interim

In [8]:
filepath = os.path.join(INTERIM_DIR, 'movies.csv')
if not os.path.exists(filepath):
    movies.to_csv(filepath, encoding='utf-8', index=False)

### tags.csv

In [9]:
tags = pd.read_csv(os.path.join(RAW_DIR, 'tags.csv'), encoding='utf-8', usecols=['movieId', 'tag'],
                   dtype={'tag': 'string'})
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   movieId  1108997 non-null  int64 
 1   tag      1108981 non-null  string
dtypes: int64(1), string(1)
memory usage: 16.9 MB


#### Pipe tags

In [10]:
tags = tags. \
    pipe(drop_na). \
    pipe(extract_stat_feature, ['movieId'], 'tag', ['count']). \
    pipe(reset_index). \
    pipe(rename, {'count':'tag_count'})

tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45981 entries, 0 to 45980
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   movieId    45981 non-null  int64
 1   tag_count  45981 non-null  int64
dtypes: int64(2)
memory usage: 718.6 KB


#### Save tags.csv to interim

In [11]:
filepath = os.path.join(INTERIM_DIR, 'tags.csv')
if not os.path.exists(filepath):
    tags.to_csv(filepath, encoding='utf-8', index=False)

### ratings.csv

In [12]:
ratings = pd.read_csv(os.path.join(RAW_DIR, 'ratings.csv'), encoding='utf-8', usecols=['movieId', 'rating'])
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   movieId  int64  
 1   rating   float64
dtypes: float64(1), int64(1)
memory usage: 423.5 MB


#### Pipe ratings

In [13]:
ratings = ratings. \
    pipe(extract_stat_feature, ['movieId'], 'rating', ['count', 'mean']). \
    pipe(reset_index). \
    pipe(rename, {'count':'rating_count', 'mean':'rating_mean'})

ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53889 entries, 0 to 53888
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       53889 non-null  int64  
 1   rating_count  53889 non-null  int64  
 2   rating_mean   53889 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.2 MB


#### Save ratings.csv to interim

In [14]:
filepath = os.path.join(INTERIM_DIR, 'ratings.csv')
if not os.path.exists(filepath):
    ratings.to_csv(filepath, encoding='utf-8', index=False)

### links.csv

In [15]:
links = pd.read_csv(os.path.join(RAW_DIR, 'links.csv'), encoding='utf-8', dtype={'imdbId': 'string'}, usecols=['movieId', 'imdbId'])
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   imdbId   58098 non-null  string
dtypes: int64(1), string(1)
memory usage: 907.9 KB


#### Pipe links

In [16]:
links = links. \
    pipe(apply, 'imdbId', lambda x: f'tt{x}'). \
    pipe(convert_to, 'imdbId', 'string')

links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   imdbId   58098 non-null  string
dtypes: int64(1), string(1)
memory usage: 907.9 KB


#### Save links.csv to interim

In [17]:
filepath = os.path.join(INTERIM_DIR, 'links.csv')
if not os.path.exists(filepath):
    links.to_csv(filepath, encoding='utf-8', index=False)